***GENERATED CODE FOR shippingpricemodel PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
from pyspark.ml.feature import StringIndexer
import json
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'OrderDate', 'transformation_label': 'Extract Date'}], 'feature': 'OrderDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
                                                'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'OrderDate'}, {'feature_label': 'OrderDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('OrderDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Carrier', 'transformation_label': 'String Indexer'}], 'feature': 'Carrier', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '13', 'mean': '', 'stddev': '', 'min': 'shopify', 'max': 'ups_shipping', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Carrier'}, {'feature_label': 'Carrier', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Carrier')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'LineItemId', 'transformation_label': 'String Indexer'}], 'feature': 'LineItemId', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '13', 'mean': '', 'stddev': '', 'min': '00346A85-ED67-49BF-8959-1B7AFD9504E3', 'max': 'FA0234BD-7E9D-466A-A1DA-7EE8739714BC', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'LineItemId'}, {'feature_label': 'LineItemId', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('LineItemId')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'OrderId', 'transformation_label': 'String Indexer'}], 'feature': 'OrderId', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '13', 'mean': '', 'stddev': '', 'min': '1BAD4BD9-D9CF-4CB1-8256-59EF94D92C4C', 'max': 'FC50F046-2D24-4284-A4FF-21BEC07BBE13', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'OrderId'}, {'feature_label': 'OrderId', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('OrderId')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProductName', 'transformation_label': 'String Indexer'}], 'feature': 'ProductName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '13', 'mean': '', 'stddev': '', 'min': 'Black Diamond 10 MM Dynex Runners - 30', 'max': 'Scarpa INSTINCT VS - 44 / Black/Orange', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProductName'}, {'feature_label': 'ProductName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProductName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SKU', 'transformation_label': 'String Indexer'}], 'feature': 'SKU', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '13', 'mean': '6670317.33', 'stddev': '5768680.37', 'min': '10000846', 'max': 'M38A SL', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SKU'}, {'feature_label': 'SKU', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SKU')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Ref_OrderLineItemId', 'transformation_label': 'String Indexer'}], 'feature': 'Ref_OrderLineItemId', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '13', 'mean': '', 'stddev': '', 'min': 'LineItemRef_5596151709857', 'max': 'LineItemRef_9851055603873', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Ref_OrderLineItemId'}, {'feature_label': 'Ref_OrderLineItemId', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Ref_OrderLineItemId')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ToZip', 'transformation_label': 'String Indexer'}], 'feature': 'ToZip', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '13', 'mean': '52760.54', 'stddev': '36015.3', 'min': '1002', 'max': '99686', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ToZip'}, {'feature_label': 'ToZip', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ToZip')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProductId', 'transformation_label': 'String Indexer'}], 'feature': 'ProductId', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '13', 'mean': '', 'stddev': '', 'min': '029967CBAF08496D880120D9D6609316', 'max': 'EC16A93A572347F5B551A8A9F773790B', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProductId'}, {'feature_label': 'ProductId', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProductId')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FulfillmentLineItemId', 'transformation_label': 'String Indexer'}], 'feature': 'FulfillmentLineItemId', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '13', 'mean': '', 'stddev': '', 'min': '12B10EF7-2774-4A60-8E42-75C76A49D9E7', 'max': 'E033E92D-1D52-4499-9F49-708730292B95', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FulfillmentLineItemId'}, {'feature_label': 'FulfillmentLineItemId', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FulfillmentLineItemId')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'OrderLineItemId', 'transformation_label': 'String Indexer'}], 'feature': 'OrderLineItemId', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '13', 'mean': '', 'stddev': '', 'min': '00346A85-ED67-49BF-8959-1B7AFD9504E3', 'max': 'FA0234BD-7E9D-466A-A1DA-7EE8739714BC', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'OrderLineItemId'}, {'feature_label': 'OrderLineItemId', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('OrderLineItemId')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FulfillmentId', 'transformation_label': 'String Indexer'}], 'feature': 'FulfillmentId', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '13', 'mean': '', 'stddev': '', 'min': '2A8D1C8C-4921-4967-954E-469F7BBB76DA', 'max': 'FF63683C-44A6-4513-A9F9-47ED44E8269E', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FulfillmentId'}, {'feature_label': 'FulfillmentId', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FulfillmentId')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'CarrierPickedUpDateTime', 'transformation_label': 'Extract Date'}], 'feature': 'CarrierPickedUpDateTime', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'CarrierPickedUpDateTime'}, {'feature_label': 'CarrierPickedUpDateTime', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('CarrierPickedUpDateTime')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'CarrierDeliveredDateTime', 'transformation_label': 'Extract Date'}], 'feature': 'CarrierDeliveredDateTime', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'CarrierDeliveredDateTime'}, {'feature_label': 'CarrierDeliveredDateTime', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop(
            'CarrierDeliveredDateTime')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run shippingpricemodelHooks.ipynb
try:
	#sourcePreExecutionHook()

	shippingpricedata = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/1630578650005_ShippingPricePrediction.csv', 'filename': '1630578650005_ShippingPricePrediction.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/BiglynxUpdated/ShippingPriceData/ShippingPricePrediction.csv', 'viewFileName': 'ShippingPricePrediction.csv', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/', 'results_url': 'http://3.140.115.35:4040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run shippingpricemodelHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(shippingpricedata,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "OrderDate", "transformation_label": "Extract Date"}], "feature": "OrderDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "OrderDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ShippingId", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "13", "mean": "1.85", "stddev": "1.63", "min": "1", "max": "5", "missing": "0"}, "updatedLabel": "ShippingId"}, {"transformationsData": [{"feature_label": "Carrier", "transformation_label": "String Indexer"}], "feature": "Carrier", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "13", "mean": "", "stddev": "", "min": "shopify", "max": "ups_shipping", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Carrier"}, {"transformationsData": [{"feature_label": "LineItemId", "transformation_label": "String Indexer"}], "feature": "LineItemId", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "13", "mean": "", "stddev": "", "min": "00346A85-ED67-49BF-8959-1B7AFD9504E3", "max": "FA0234BD-7E9D-466A-A1DA-7EE8739714BC", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "LineItemId"}, {"transformationsData": [{"feature_label": "OrderId", "transformation_label": "String Indexer"}], "feature": "OrderId", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "13", "mean": "", "stddev": "", "min": "1BAD4BD9-D9CF-4CB1-8256-59EF94D92C4C", "max": "FC50F046-2D24-4284-A4FF-21BEC07BBE13", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "OrderId"}, {"transformationsData": [{"feature_label": "ProductName", "transformation_label": "String Indexer"}], "feature": "ProductName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "13", "mean": "", "stddev": "", "min": "Black Diamond 10 MM Dynex Runners - 30", "max": "Scarpa INSTINCT VS - 44 / Black/Orange", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProductName"}, {"transformationsData": [{"feature_label": "SKU", "transformation_label": "String Indexer"}], "feature": "SKU", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "13", "mean": "6670317.33", "stddev": "5768680.37", "min": "10000846", "max": "M38A SL", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SKU"}, {"transformationsData": [{"feature_label": "Ref_OrderLineItemId", "transformation_label": "String Indexer"}], "feature": "Ref_OrderLineItemId", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "13", "mean": "", "stddev": "", "min": "LineItemRef_5596151709857", "max": "LineItemRef_9851055603873", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Ref_OrderLineItemId"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Qty", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "13", "mean": "1.08", "stddev": "0.28", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "Qty"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Length", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "13", "mean": "3.31", "stddev": "2.36", "min": "0", "max": "8", "missing": "0"}, "updatedLabel": "Length"}, {"transformationsData": [{"feature_label": "ToZip", "transformation_label": "String Indexer"}], "feature": "ToZip", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "13", "mean": "52760.54", "stddev": "36015.3", "min": "1002", "max": "99686", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ToZip"}, {"transformationsData": [{"feature_label": "ProductId", "transformation_label": "String Indexer"}], "feature": "ProductId", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "13", "mean": "", "stddev": "", "min": "029967CBAF08496D880120D9D6609316", "max": "EC16A93A572347F5B551A8A9F773790B", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProductId"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FromZip", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "13", "mean": "98052.0", "stddev": "0.0", "min": "98052", "max": "98052", "missing": "0"}, "updatedLabel": "FromZip"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RetailerId", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "13", "mean": "375544787.0", "stddev": "0.0", "min": "375544787", "max": "375544787", "missing": "0"}, "updatedLabel": "RetailerId"}, {"transformationsData": [{"feature_label": "FulfillmentLineItemId", "transformation_label": "String Indexer"}], "feature": "FulfillmentLineItemId", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "13", "mean": "", "stddev": "", "min": "12B10EF7-2774-4A60-8E42-75C76A49D9E7", "max": "E033E92D-1D52-4499-9F49-708730292B95", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FulfillmentLineItemId"}, {"transformationsData": [{"feature_label": "OrderLineItemId", "transformation_label": "String Indexer"}], "feature": "OrderLineItemId", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "13", "mean": "", "stddev": "", "min": "00346A85-ED67-49BF-8959-1B7AFD9504E3", "max": "FA0234BD-7E9D-466A-A1DA-7EE8739714BC", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "OrderLineItemId"}, {"transformationsData": [{"feature_label": "FulfillmentId", "transformation_label": "String Indexer"}], "feature": "FulfillmentId", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "13", "mean": "", "stddev": "", "min": "2A8D1C8C-4921-4967-954E-469F7BBB76DA", "max": "FF63683C-44A6-4513-A9F9-47ED44E8269E", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FulfillmentId"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "quantity", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "13", "mean": "1.08", "stddev": "0.28", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "quantity"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Weight", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "13", "mean": "0.99", "stddev": "1.06", "min": "0.0", "max": "3.04", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Weight"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Volume", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "13", "mean": "163.38", "stddev": "176.04", "min": "0", "max": "504", "missing": "0"}, "updatedLabel": "Volume"}, {"transformationsData": [{"feature_label": "CarrierPickedUpDateTime", "transformation_label": "Extract Date"}], "feature": "CarrierPickedUpDateTime", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "CarrierPickedUpDateTime"}, {"transformationsData": [{"feature_label": "CarrierDeliveredDateTime", "transformation_label": "Extract Date"}], "feature": "CarrierDeliveredDateTime", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "CarrierDeliveredDateTime"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ShipCost", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "13", "mean": "3.92", "stddev": "3.35", "min": "0", "max": "10", "missing": "0"}, "updatedLabel": "ShipCost"}, {"feature": "OrderDate_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "13", "mean": "13.31", "stddev": "6.63", "min": "7", "max": "28", "missing": "0"}, "updatedLabel": "OrderDate_dayofmonth"}, {"feature": "OrderDate_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "13", "mean": "8.08", "stddev": "3.04", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "OrderDate_month"}, {"feature": "OrderDate_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "13", "mean": "2020.08", "stddev": "0.28", "min": "2020", "max": "2021", "missing": "0"}, "updatedLabel": "OrderDate_year"}, {"feature": "Carrier_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "13", "mean": "0.08", "stddev": "0.28", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Carrier_stringindexer_tra..."}, {"feature": "LineItemId_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "13", "mean": "6.0", "stddev": "3.89", "min": "0.0", "max": "12.0", "missing": "0"}, "updatedLabel": "LineItemId_stringindexer_..."}, {"feature": "OrderId_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "13", "mean": "5.08", "stddev": "3.77", "min": "0.0", "max": "11.0", "missing": "0"}, "updatedLabel": "OrderId_stringindexer_tra..."}, {"feature": "ProductName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "13", "mean": "6.0", "stddev": "3.89", "min": "0.0", "max": "12.0", "missing": "0"}, "updatedLabel": "ProductName_stringindexer..."}, {"feature": "SKU_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "13", "mean": "6.0", "stddev": "3.89", "min": "0.0", "max": "12.0", "missing": "0"}, "updatedLabel": "SKU_stringindexer_transfo..."}, {"feature": "Ref_OrderLineItemId_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "13", "mean": "6.0", "stddev": "3.89", "min": "0.0", "max": "12.0", "missing": "0"}, "updatedLabel": "Ref_OrderLineItemId_strin..."}, {"feature": "ToZip_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "13", "mean": "5.08", "stddev": "3.77", "min": "0.0", "max": "11.0", "missing": "0"}, "updatedLabel": "ToZip_stringindexer_trans..."}, {"feature": "ProductId_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "13", "mean": "6.0", "stddev": "3.89", "min": "0.0", "max": "12.0", "missing": "0"}, "updatedLabel": "ProductId_stringindexer_t..."}, {"feature": "FulfillmentLineItemId_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "13", "mean": "6.0", "stddev": "3.89", "min": "0.0", "max": "12.0", "missing": "0"}, "updatedLabel": "FulfillmentLineItemId_str..."}, {"feature": "OrderLineItemId_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "13", "mean": "6.0", "stddev": "3.89", "min": "0.0", "max": "12.0", "missing": "0"}, "updatedLabel": "OrderLineItemId_stringind..."}, {"feature": "FulfillmentId_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "13", "mean": "6.0", "stddev": "3.89", "min": "0.0", "max": "12.0", "missing": "0"}, "updatedLabel": "FulfillmentId_stringindex..."}, {"feature": "CarrierPickedUpDateTime_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "13", "mean": "9.46", "stddev": "1.98", "min": "8", "max": "16", "missing": "0"}, "updatedLabel": "CarrierPickedUpDateTime_d..."}, {"feature": "CarrierPickedUpDateTime_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "13", "mean": "3.08", "stddev": "2.66", "min": "2", "max": "10", "missing": "0"}, "updatedLabel": "CarrierPickedUpDateTime_m..."}, {"feature": "CarrierPickedUpDateTime_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "13", "mean": "2020.85", "stddev": "0.38", "min": "2020", "max": "2021", "missing": "0"}, "updatedLabel": "CarrierPickedUpDateTime_y..."}, {"feature": "CarrierDeliveredDateTime_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "13", "mean": "10.23", "stddev": "3.24", "min": "9", "max": "20", "missing": "0"}, "updatedLabel": "CarrierDeliveredDateTime_..."}, {"feature": "CarrierDeliveredDateTime_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "13", "mean": "3.77", "stddev": "1.88", "min": "3", "max": "8", "missing": "0"}, "updatedLabel": "CarrierDeliveredDateTime_..."}, {"feature": "CarrierDeliveredDateTime_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "13", "mean": "2020.85", "stddev": "0.38", "min": "2020", "max": "2021", "missing": "0"}, "updatedLabel": "CarrierDeliveredDateTime_..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run shippingpricemodelHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(autofe, ["ShippingId", "Carrier_stringindexer", "LineItemId_stringindexer", "OrderId_stringindexer", "ProductName_stringindexer", "SKU_stringindexer", "Ref_OrderLineItemId_stringindexer", "Qty", "Length", "ToZip_stringindexer", "ProductId_stringindexer", "FromZip", "RetailerId", "FulfillmentLineItemId_stringindexer", "OrderLineItemId_stringindexer", "FulfillmentId_stringindexer", "quantity", "Weight", "Volume", "OrderDate_dayofmonth", "OrderDate_month", "OrderDate_year", "CarrierPickedUpDateTime_dayofmonth", "CarrierPickedUpDateTime_month", "CarrierPickedUpDateTime_year", "CarrierDeliveredDateTime_dayofmonth", "CarrierDeliveredDateTime_month", "CarrierDeliveredDateTime_year"], "ShipCost")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

